In [1]:
import gym
import numpy as np

In [2]:
env = gym.make("CartPole-v0")

[2017-08-11 11:45:49,662] Making new env: CartPole-v0


In [3]:
obs = env.reset()

In [4]:
obs

array([ 0.04858836, -0.0287559 ,  0.03665763,  0.04530748])

In [5]:
env.action_space

Discrete(2)

In [6]:
action = 1
obs, reward, done, info = env.step(action)

In [7]:
obs

array([ 0.04801324,  0.16582175,  0.03756378, -0.23558798])

In [8]:
reward

1.0

In [9]:
done

False

In [10]:
info

{}

In [11]:
action = 0

In [12]:
obs, reward, done, info = env.step(action)

In [13]:
obs

array([ 0.05132967, -0.02981619,  0.03285202,  0.06870307])

In [14]:
obs, reward, done, info = env.step(action)

In [15]:
obs

array([ 0.05073335, -0.22539336,  0.03422608,  0.37156728])

In [16]:
import tensorflow as tf

In [17]:
from tensorflow.contrib.layers import fully_connected

In [18]:
n_inputs = 4
n_hidden = 4
n_outputs = 1

In [19]:
initializer = tf.contrib.layers.variance_scaling_initializer()

In [20]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs))

In [21]:
hidden = fully_connected(X, n_hidden, activation_fn=tf.nn.elu, weights_initializer=initializer)
logits = fully_connected(hidden, n_outputs, activation_fn=None, weights_initializer=initializer)
outputs = tf.nn.sigmoid(logits)

In [22]:
p_l_r = tf.concat(axis=1, values=[outputs, 1-outputs])
action = tf.multinomial(tf.log(p_l_r), num_samples=1)
init = tf.global_variables_initializer()

In [23]:
y = 1. - tf.to_float(action)

In [24]:
learning_rate = 0.01

cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(
                    labels=y, logits=logits)
optimizer = tf.train.AdamOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(cross_entropy)

In [25]:
gradients = [grad for grad, variable in grads_and_vars]

In [26]:
gradient_placeholders = []
grads_and_vars_feed = []
for grad, variable in grads_and_vars:
    gradient_placeholder = tf.placeholder(tf.float32, shape=grad.get_shape())
    gradient_placeholders.append(gradient_placeholder)
    grads_and_vars_feed.append((gradient_placeholder, variable))

training_op = optimizer.apply_gradients(grads_and_vars_feed)

In [31]:
def discount_rewards(rewards, discount_rate):
    discounted_rewards = np.empty(len(rewards))
    cumulative_rewards = 0
    for step in reversed(range(len(rewards))):
        cumulative_rewards = rewards[step] + cumulative_rewards * discount_rate
        discounted_rewards[step] = cumulative_rewards
    return discounted_rewards

def discount_and_normalize_rewards(all_rewards, discount_rate):
    all_discounted_rewards = [discount_rewards(rewards, discount_rate)
                              for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean)/reward_std
            for discounted_rewards in all_discounted_rewards]

In [32]:
discount_rewards([10, 0, -50], discount_rate=0.8)

array([-22., -40., -50.])

In [33]:
n_iterations = 250      # number of training iterations
n_max_steps = 1000      # max steps per episode
n_games_per_update = 10 # train the policy every 10 episodes
save_iterations = 10    # save the model every 10 training iterations
discount_rate = 0.95

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        all_rewards = []    # all sequences of raw rewards for each episode
        all_gradients = []  # gradients saved at each step of each episode
        for game in range(n_games_per_update):
            current_rewards = []   # all raw rewards from the current episode
            current_gradients = [] # all gradients from the current episode
            obs = env.reset()
            for step in range(n_max_steps):
                action_val, gradients_val = sess.run(
                        [action, gradients],
                        feed_dict={X: obs.reshape(1, n_inputs)}) # one obs
                obs, reward, done, info = env.step(action_val[0][0])
                current_rewards.append(reward)
                current_gradients.append(gradients_val)
                if done:
                    break
            all_rewards.append(current_rewards)
            all_gradients.append(current_gradients)

        # At this point we have run the policy for 10 episodes, and we are
        # ready for a policy update using the algorithm described earlier.
        all_rewards = discount_and_normalize_rewards(all_rewards, discount_rate)
        feed_dict = {}
        for var_index, grad_placeholder in enumerate(gradient_placeholders):
            # multiply the gradients by the action scores, and compute the mean
            mean_gradients = np.mean(
                [reward * all_gradients[game_index][step][var_index]
                    for game_index, rewards in enumerate(all_rewards)
                    for step, reward in enumerate(rewards)],
                axis=0)
            feed_dict[grad_placeholder] = mean_gradients
        sess.run(training_op, feed_dict=feed_dict)
        if iteration % save_iterations == 0:
            saver.save(sess, "./my_policy_net_pg.ckpt")

FailedPreconditionError: Attempting to use uninitialized value beta1_power
	 [[Node: beta1_power/read = Identity[T=DT_FLOAT, _class=["loc:@fully_connected/weights"], _device="/job:localhost/replica:0/task:0/gpu:0"](beta1_power)]]

Caused by op 'beta1_power/read', defined at:
  File "C:\Users\liviu\Anaconda3\envs\ztdlg\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\liviu\Anaconda3\envs\ztdlg\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\liviu\Anaconda3\envs\ztdlg\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\liviu\Anaconda3\envs\ztdlg\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\liviu\Anaconda3\envs\ztdlg\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\liviu\Anaconda3\envs\ztdlg\lib\site-packages\zmq\eventloop\